In [ ]:
import sys
import os

# 프로젝트 루트 추가
project_root = os.path.dirname(os.path.abspath("__file__")) if "__file__" in globals() else os.getcwd()
if project_root not in sys.path:
    sys.path.insert(0, project_root)

import time
import functools
def benchmark_runtime_decorator(func):
    @functools.wraps(func)
    def wrapper_benchmark_runtime(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        elapsed_time = end_time - start_time
        elapsed_minute = int(elapsed_time / 60) if elapsed_time >= 60 else None
        print(f"Function '{func.__name__}' executed in {f"{elapsed_minute} minutes " if elapsed_minute else ""}{elapsed_time:.4f} seconds")
        return result

    return wrapper_benchmark_runtime

In [ ]:
from common.Generator import Generator
from common.test_backendclient import TestBackendClient as BackendClient
import logging

# 기존 핸들러 제거
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

# 기본 로깅 설정
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout),
    ]
)

# 루트 로거 레벨
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

backend_client = BackendClient()
generator = Generator()

generator._test_pdf_create(backend_client)

In [ ]:
from common.Generator import Generator
from common.test_backendclient import TestBackendClient as BackendClient
from test.test_data_loader import test_data_loader_v1


backend_client = BackendClient()
generator = Generator()

# task_id = "session-20251002-052932-151e52e9" # 10만개 데이터
task_id = "session-20251002-040520-28a9dc6c" # 정호주임 pc

job_id = "73c94dcd-d1a1-41c4-9636-dd24816dfd9a"

@benchmark_runtime_decorator
def load_artifacts():
    artifacts = test_data_loader_v1(task_id)
    print(f"\n✅ 총 {len(artifacts):,}개 아티팩트 로드 완료")
    return artifacts

artifacts = load_artifacts()

job_info = backend_client.load_job_info(task_id, job_id)
print(job_info)

In [ ]:
@benchmark_runtime_decorator
def generate_report(generator):
    generator.generate_report(task_id, job_id, artifacts, job_info, backend_client)

generate_report(generator)

In [ ]:
for item in generator.report:
    if item[0] == 'details':
        title, data = item
        for i in data:
            print(i.section_type.value, i.content)
        continue
    print(item)